In [1]:
import sys
sys.path.insert(0, '/home/ldoyle/packages')
import h5py
import numpy as np
from tqdm import tqdm
import matplotlib
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import pandas as pd
import pysindy as ps
from lr_ed import localreg
from datetime import datetime
from scipy.fft import fft, fftfreq,rfft2

import scipy.signal as signal
from obspy.signal import filter as obsfilt
import glob


In [2]:
WEAK = False

x_len = 5000
t_len = 6000

xs = np.arange(x_len)
dt = 1

library_functions = [lambda x: x, lambda x: x * x]
library_function_names = [lambda x: x, lambda x: x + x]  

if WEAK:
    X, T = np.meshgrid(xs, np.arange(t_len))
    XT = np.asarray([X, T]).T

    pde_lib = ps.WeakPDELibrary(
        library_functions=library_functions,
        function_names=library_function_names,
        derivative_order=4,
        spatiotemporal_grid=XT,
        is_uniform=True,
        K=1000,
    )
else:
    
    pde_lib = ps.PDELibrary(
        library_functions=library_functions,
        function_names=library_function_names,
        derivative_order=4,
        spatial_grid=xs,
        include_bias=True,
        is_uniform=True,
    )
    

In [3]:
def printEnsemble(model, cut_off = 1e-3, median= False):
    if median:
        coefs = np.median(model.coef_list, axis=0)[0,:]
    else:
        coefs = np.mean(model.coef_list, axis=0)[0,:]
    above_cut_off = np.argwhere(np.abs(coefs)>cut_off).flatten()
    features =  model.get_feature_names()
    equation_str= "(x0)' = "
    for count, val in enumerate(above_cut_off):
        if count >0:
            equation_str+=" + "
        equation_str+= "{0:.4f}".format(coefs[val])
        equation_str+= " "+features[val]
        
    print(equation_str)

In [4]:
def saveEnsemble(model,fileStr,thresh):
    np.save("/home/ldoyle/notebooks/ensembleSINDyArrays_50/"+fileStr+"_thresh_"+"{:.2E}".format(thresh), model.coef_list)
#     print(fileStr+"_thresh_"+"{:.2E}".format(thresh))

In [5]:
def SINDyImplementEnsemble(dataset, fileStr,thresh, alph=0.001, n_models=50, time_steps=200, stlsq_max =100,median=True):
    optimizer = ps.STLSQ(threshold=thresh, alpha=alph, max_iter=stlsq_max)
#     optimizer = ps.EnsembleOptimizer( opt=ps.STLSQ(threshold=thresh, alpha=alph, max_iter=stlsq_max),bagging=True,  n_models = n_models,n_subset =time_steps)  
    model = ps.SINDy(feature_library=pde_lib, optimizer=optimizer, 
                     differentiation_method= ps.differentiation.SmoothedFiniteDifference())
    model.fit(dataset, t=dt,ensemble=True, n_models=n_models, n_subset=time_steps)
    print("Ensemble STLSQ with Threshold "+str(thresh))
    printEnsemble(model,median=median)
    saveEnsemble(model,fileStr,thresh)

## Unfiltered

### Dataset 1

In [6]:
path = "/data/data2/south-data-ejm/hdd/South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T16_09_15-0700/"

fileStr = "2021-11-01T2311_unfiltered"

init = 11
final= 16

time_subsample = 10
full_dat = np.zeros((5000,(12000//time_subsample)*(final-init)))

k = 0
for i in np.arange(init,final):
    file = "South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T23"+str(i)+"14Z.h5"
    f = h5py.File(path+file, 'r')
    data = f['Acquisition']['Raw[0]']['RawData'][:, :].astype('int64')
#     timestamp = f['Acquisition']['Raw[0]']['RawDataTime'][:] / 1000000
    f.close()
    
    full_dat[:,k*(12000//time_subsample):(k+1)*(12000//time_subsample)] = data[7500:12500,::time_subsample]
    k+=1
    
working_dat = full_dat/np.std(full_dat)


In [7]:
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [8]:
thresh = 1.e-1
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.1
(x0)' = 0.6367 x0_1 + 0.3535 x0_111


In [9]:
thresh = 1.e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.6400 x0_1 + 0.3545 x0_111


In [10]:
thresh = 5e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.005
(x0)' = 0.6459 x0_1 + 0.3556 x0_111 + 0.0148 x0x0_1


In [11]:
thresh = 3e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.003
(x0)' = 0.6332 x0_1 + 0.0055 x0_11 + 0.3481 x0_111 + 0.0019 x0_1111 + 0.0150 x0x0_1 + 0.0123 x0x0x0_1 + -0.0200 x0x0_11 + 0.0112 x0x0x0_111


In [12]:
thresh = 1e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.001
(x0)' = 0.0018 1 + -0.0015 x0 + 0.6589 x0_1 + -0.0019 x0_11 + 0.3610 x0_111 + 0.0012 x0_1111 + 0.0080 x0x0_1 + 0.0014 x0x0x0_1 + 0.0043 x0x0_11 + -0.0044 x0x0x0_11 + -0.0082 x0x0_111 + 0.0021 x0x0x0_111 + -0.0024 x0x0x0_1111


### Dataset 2

In [13]:
time_subsample = 10
full_dat = np.zeros((5000,6000))
fileStr = "2021-11-01T2321_unfiltered"


k = 0
for i in np.arange(21,26):
    file = "South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T23"+str(i)+"14Z.h5"
    f = h5py.File(path+file, 'r')
    data = f['Acquisition']['Raw[0]']['RawData'][:, :].astype('int64')
#     timestamp = f['Acquisition']['Raw[0]']['RawDataTime'][:] / 1000000
    f.close()
    
    full_dat[:,k*(12000//time_subsample):(k+1)*(12000//time_subsample)] = data[7500:12500,::time_subsample]
    k+=1
    


In [14]:
working_dat = full_dat/np.std(full_dat)#*10**6
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [15]:
thresh = 1.e-1
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.1
(x0)' = 0.6766 x0_1 + 0.3854 x0_111


In [16]:
thresh = 1.e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.6654 x0_1 + 0.3921 x0_111


In [17]:
thresh = 5e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.005
(x0)' = 0.6732 x0_1 + -0.0127 x0_11 + 0.3880 x0_111 + -0.0027 x0_1111 + 0.0057 x0x0x0_111


In [18]:
thresh = 3e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.003
(x0)' = -0.0072 x0x0 + 0.6705 x0_1 + -0.0113 x0_11 + 0.3812 x0_111 + -0.0015 x0_1111 + 0.0026 x0x0x0_1 + -0.0087 x0x0_11 + 0.0050 x0x0x0_11 + -0.0147 x0x0_111 + 0.0062 x0x0x0_111 + -0.0019 x0x0_1111


In [19]:
thresh = 1e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.001
(x0)' = 0.0014 1 + 0.0011 x0 + -0.0053 x0x0 + 0.6699 x0_1 + 0.0021 x0_11 + 0.3795 x0_111 + -0.0065 x0x0_1 + 0.0032 x0x0x0_1 + 0.0042 x0x0x0_11 + -0.0096 x0x0_111 + 0.0014 x0x0x0_111 + 0.0017 x0x0_1111 + 0.0028 x0x0x0_1111


## Drift Removal Load

### Dataset 1

In [6]:
fileStr = "2021-11-01T2311_normal"

working_dat = np.load("/home/ldoyle/notebooks/channel_drift_5_min_clips/normalized_5_min_start_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T231114Z.npy")[:,::10]
scaler_val = np.std(working_dat)
working_dat = working_dat/scaler_val
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [7]:
thresh = 1e-1
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.1
(x0)' = 0.7000 x0_1 + 0.3278 x0_111


In [8]:
thresh = 1e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.6960 x0_1 + 0.3407 x0_111 + -0.0195 x0x0x0_1


In [11]:
thresh = 5e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.005
(x0)' = 0.7027 x0_1 + 0.0143 x0_11 + 0.3393 x0_111 + -0.0097 x0x0x0_1 + -0.0078 x0x0x0_111


In [9]:
thresh = 3e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.003
(x0)' = 0.6934 x0_1 + 0.0018 x0_11 + 0.3269 x0_111 + 0.0165 x0x0_1 + -0.0118 x0x0x0_1 + 0.0045 x0x0_111 + -0.0074 x0x0x0_111


In [10]:
thresh = 1e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.001
(x0)' = 0.7187 x0_1 + 0.0058 x0_11 + 0.3341 x0_111 + 0.0018 x0_1111 + 0.0042 x0x0_1 + -0.0228 x0x0x0_1 + -0.0020 x0x0_11 + 0.0107 x0x0x0_11 + 0.0035 x0x0_111 + -0.0083 x0x0x0_111 + -0.0020 x0x0_1111 + 0.0028 x0x0x0_1111


### Dataset 2

In [12]:
fileStr = "2021-11-01T2321_normal"

working_dat = np.load("/home/ldoyle/notebooks/channel_drift_5_min_clips/normalized_5_min_start_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T232114Z.npy")[:,::10]
scaler_val = np.std(working_dat)
working_dat = working_dat/scaler_val
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [13]:
thresh = 1e-1
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.1
(x0)' = 0.6663 x0_1 + 0.3045 x0_111


In [14]:
thresh = 1e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.6852 x0_1 + 0.3322 x0_111


In [15]:
thresh = 5e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.005
(x0)' = 0.6836 x0_1 + 0.3269 x0_111


In [16]:
thresh = 3e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.003
(x0)' = 0.6725 x0_1 + 0.0098 x0_11 + 0.3158 x0_111 + 0.0022 x0_1111 + 0.0121 x0x0x0_1 + 0.0081 x0x0_111 + 0.0098 x0x0x0_111


In [17]:
thresh = 1e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.001
(x0)' = 0.6696 x0_1 + -0.0069 x0_11 + 0.3162 x0_111 + -0.0029 x0_1111 + 0.0092 x0x0_1 + -0.0031 x0x0x0_11


### Dataset 3

In [18]:
fileStr = "2021-11-02T0511_normal"

working_dat = np.load("/home/ldoyle/notebooks/channel_drift_5_min_clips/normalized_5_min_start_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-02T051114Z.npy")[:,::10]
scaler_val = np.std(working_dat)
working_dat = working_dat/scaler_val
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [19]:
thresh = 1e-1
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.1
(x0)' = 0.6797 x0_1 + 0.3213 x0_111


In [20]:
thresh = 1e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.6773 x0_1 + 0.3271 x0_111


In [21]:
thresh = 5e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.005
(x0)' = 0.6950 x0_1 + 0.3280 x0_111


In [22]:
thresh = 3e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.003
(x0)' = 0.6946 x0_1 + 0.3322 x0_111 + 0.0019 x0x0_1 + 0.0117 x0x0_11 + 0.0015 x0x0x0_11 + -0.0050 x0x0_111 + 0.0019 x0x0_1111


In [23]:
thresh = 1e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.001
(x0)' = 0.6835 x0_1 + 0.0121 x0_11 + 0.3295 x0_111 + 0.0035 x0_1111 + 0.0091 x0x0_1 + -0.0017 x0x0x0_1 + -0.0079 x0x0_11 + -0.0049 x0x0x0_11 + 0.0054 x0x0_111 + -0.0030 x0x0x0_111


### Dataset 4

In [24]:
fileStr = "2021-11-02T2311_normal"

working_dat = np.load("/home/ldoyle/notebooks/channel_drift_5_min_clips/normalized_5_min_start_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-02T231114Z.npy")[:,::10]
scaler_val = np.std(working_dat)
working_dat = working_dat/scaler_val
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [25]:
thresh = 1e-1
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.1
(x0)' = 0.6159 x0_1 + 0.2858 x0_111


In [26]:
thresh = 1e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.6312 x0_1 + 0.3030 x0_111


In [27]:
thresh = 5e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.005
(x0)' = 0.5899 x0_1 + 0.0078 x0_11 + 0.2803 x0_111 + -0.0172 x0x0_11


In [28]:
thresh = 3e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.003
(x0)' = 0.6310 x0_1 + -0.0141 x0_11 + 0.2936 x0_111 + 0.0152 x0x0_1 + 0.0160 x0x0_11 + 0.0058 x0x0_111 + 0.0047 x0x0_1111


In [29]:
thresh = 1e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.001
(x0)' = 0.6383 x0_1 + 0.0098 x0_11 + 0.2985 x0_111 + 0.0021 x0_1111 + 0.0162 x0x0_1 + -0.0118 x0x0x0_1 + -0.0023 x0x0_11 + 0.0032 x0x0x0_11 + 0.0152 x0x0_111 + -0.0056 x0x0x0_111 + -0.0046 x0x0_1111


## Multifilt

### Dataset 1

In [6]:
fileStr = "2021-11-01T2311_smoothed_rad_4"

working_dat = np.load("/home/ldoyle/notebooks/channel_drift_5_min_clips/rad_4_smoothed_decimated_normalized_5_min_start_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T231114Z.npy")
scaler_val = np.std(working_dat)
working_dat = working_dat/scaler_val
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [7]:
thresh = 1e-1
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.1
(x0)' = 0.9937 x0_1


In [8]:
thresh = 5e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.05
(x0)' = 0.9938 x0_1


In [9]:
thresh = 1e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.9944 x0_1 + 0.0475 x0_111 + 0.0450 x0x0x0_111


In [10]:
thresh = 3e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.003
(x0)' = 0.9921 x0_1 + 0.0469 x0_111 + 0.0032 x0x0x0_1 + 0.0454 x0x0x0_111


In [11]:
thresh = 1e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.001
(x0)' = 0.9912 x0_1 + 0.0466 x0_111 + 0.0033 x0x0x0_1 + 0.0489 x0x0x0_111


### Dataset 2

In [12]:
fileStr = "2021-11-01T2321_smoothed_rad_4"

working_dat = np.load("/home/ldoyle/notebooks/channel_drift_5_min_clips/rad_4_smoothed_decimated_normalized_5_min_start_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T232114Z.npy")
scaler_val = np.std(working_dat)
working_dat = working_dat/scaler_val
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [13]:
thresh = 1e-1
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.1
(x0)' = 0.9939 x0_1


In [14]:
thresh = 5e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.05
(x0)' = 0.9943 x0_1 + 0.0599 x0_111


In [15]:
thresh = 1e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.9947 x0_1 + 0.0550 x0_111 + 0.0472 x0x0x0_111


In [16]:
thresh = 3e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.003
(x0)' = 0.9943 x0_1 + 0.0544 x0_111 + 0.0478 x0x0x0_111


In [17]:
thresh = 1e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.001
(x0)' = 0.9919 x0_1 + 0.0542 x0_111 + 0.0027 x0x0x0_1 + 0.0525 x0x0x0_111


### Dataset 3

In [18]:
fileStr = "2021-11-02T0511_smoothed_rad_4"

working_dat = np.load("/home/ldoyle/notebooks/channel_drift_5_min_clips/rad_4_smoothed_decimated_normalized_5_min_start_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-02T051114Z.npy")
scaler_val = np.std(working_dat)
working_dat = working_dat/scaler_val
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [19]:
thresh = 1e-1
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.1
(x0)' = 0.9942 x0_1


In [20]:
thresh = 5e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.05
(x0)' = 0.9944 x0_1 + 0.0645 x0_111


In [21]:
thresh = 1e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.9949 x0_1 + 0.0578 x0_111 + 0.0445 x0x0x0_111


In [22]:
thresh = 3e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.003
(x0)' = 0.9947 x0_1 + 0.0579 x0_111 + 0.0458 x0x0x0_111


In [23]:
thresh = 1e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.001
(x0)' = 0.9924 x0_1 + 0.0563 x0_111 + 0.0023 x0x0x0_1 + 0.0502 x0x0x0_111


### Dataset 4

In [24]:
fileStr = "2021-11-02T2311_smoothed_rad_4"

working_dat = np.load("/home/ldoyle/notebooks/channel_drift_5_min_clips/rad_4_smoothed_decimated_normalized_5_min_start_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-02T231114Z.npy")
scaler_val = np.std(working_dat)
working_dat = working_dat/scaler_val
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [25]:
thresh = 1e-1
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.1
(x0)' = 0.9957 x0_1 + 0.1386 x0_111


In [26]:
thresh = 5e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.05
(x0)' = 0.9961 x0_1 + 0.1142 x0_111 + 0.0625 x0x0x0_111


In [27]:
thresh = 1e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.9963 x0_1 + 0.1119 x0_111 + 0.0615 x0x0x0_111


In [28]:
thresh = 3e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.003
(x0)' = 0.9960 x0_1 + 0.1103 x0_111 + 0.0636 x0x0x0_111


In [29]:
thresh = 1e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.001
(x0)' = 0.9939 x0_1 + 0.1089 x0_111 + 0.0021 x0x0x0_1 + 0.0714 x0x0x0_111


## Multifilt (radius 10 smooth)

### Dataset 1

In [39]:
fileStr = "2021-11-01T2311_smoothed_rad_10"

working_dat = np.load("/home/ldoyle/notebooks/channel_drift_5_min_clips/rad_10_smoothed_decimated_normalized_5_min_start_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T231114Z.npy")
scaler_val = np.std(working_dat)
working_dat = working_dat/scaler_val
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [40]:
thresh = 1e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.9951 x0_1


In [ ]:
thresh = 3e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

In [41]:
thresh = 1e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.001
(x0)' = 0.9922 x0_1 + -0.0012 x0_111 + 0.0033 x0x0x0_1


In [42]:
thresh = 1e-4
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.0001
(x0)' = 0.9920 x0_1 + -0.0012 x0_111 + 0.0034 x0x0x0_1


In [43]:
thresh = 1e-5
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 1e-05
(x0)' = 0.9917 x0_1 + -0.0012 x0_111 + 0.0036 x0x0x0_1


### Dataset 2

In [44]:
fileStr = "2021-11-01T2321_smoothed_rad_10"

working_dat = np.load("/home/ldoyle/notebooks/channel_drift_5_min_clips/rad_10_smoothed_decimated_normalized_5_min_start_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T232114Z.npy")
scaler_val = np.std(working_dat)
working_dat = working_dat/scaler_val
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [45]:
thresh = 1e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.9955 x0_1


In [ ]:
thresh = 3e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

In [46]:
thresh = 1e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.001
(x0)' = 0.9930 x0_1 + -0.0014 x0_111 + 0.0025 x0x0x0_1


In [47]:
thresh = 1e-4
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.0001
(x0)' = 0.9929 x0_1 + -0.0014 x0_111 + 0.0026 x0x0x0_1


In [48]:
thresh = 1e-5
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 1e-05
(x0)' = 0.9928 x0_1 + -0.0014 x0_111 + 0.0027 x0x0x0_1


### Dataset 3

In [49]:
fileStr = "2021-11-02T0511_smoothed_rad_10"

working_dat = np.load("/home/ldoyle/notebooks/channel_drift_5_min_clips/rad_10_smoothed_decimated_normalized_5_min_start_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-02T051114Z.npy")
scaler_val = np.std(working_dat)
working_dat = working_dat/scaler_val
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [50]:
thresh = 1e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.9957 x0_1


In [ ]:
thresh = 3e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

In [51]:
thresh = 1e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.001
(x0)' = 0.9937 x0_1 + -0.0014 x0_111 + 0.0020 x0x0x0_1


In [52]:
thresh = 1e-4
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.0001
(x0)' = 0.9936 x0_1 + -0.0014 x0_111 + 0.0021 x0x0x0_1


In [53]:
thresh = 1e-5
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 1e-05
(x0)' = 0.9934 x0_1 + -0.0014 x0_111 + 0.0022 x0x0x0_1


### Dataset 4

In [54]:
fileStr = "2021-11-02T2311_smoothed_rad_10"

working_dat = np.load("/home/ldoyle/notebooks/channel_drift_5_min_clips/rad_10_smoothed_decimated_normalized_5_min_start_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-02T231114Z.npy")
scaler_val = np.std(working_dat)
working_dat = working_dat/scaler_val
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [55]:
thresh = 1e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.9970 x0_1


In [ ]:
thresh = 3e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

In [56]:
thresh = 1e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.001
(x0)' = 0.9953 x0_1 + -0.0021 x0_111 + 0.0015 x0x0x0_1


In [57]:
thresh = 1e-4
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.0001
(x0)' = 0.9953 x0_1 + -0.0021 x0_111 + 0.0016 x0x0x0_1


In [58]:
thresh = 1e-5
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 1e-05
(x0)' = 0.9952 x0_1 + -0.0021 x0_111 + 0.0017 x0x0x0_1
